In [17]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import datetime as dt
import humanize

In [22]:
def duration_to_string(x):
    if x!= x:
        return
    if x < 1:
        return humanize.precisedelta(dt.timedelta(seconds=x), minimum_unit='milliseconds').replace('milliseconds', 'msecs')
    elif x < 60:
        return humanize.precisedelta(dt.timedelta(seconds=x), minimum_unit='seconds').replace('seconds', 'secs')
    else:
        return humanize.precisedelta(dt.timedelta(seconds=x), minimum_unit='minutes').replace('minutes', 'mins')


In [23]:
results_paths = ['network/report/benchs.json']

In [24]:
benches = pd.read_json(*results_paths, orient='records')
benches = benches.rename(columns={
    'backend': 'Backend Engine',
    'operation': 'Query',
    'dataset': 'Dataset',
    'iterations': 'Iterations',
    'dataset_size': 'Dataset Size, bytes',
    'seconds': 'Benchmark Duration, seconds',
})
benches['Iteration Duration, seconds'] = benches['Benchmark Duration, seconds'] / benches['Iterations']
benches['Iteration Duration'] = benches['Iteration Duration, seconds'].apply(duration_to_string)


In [25]:
for name, group in benches.groupby(['Query']):
    if name == 'Close':
        continue

    # Docs:
    # https://plotly.com/python/bar-charts/
    fig = px.bar(
        group, 
        title=name,
        x='Dataset', 
        y='Iteration Duration, seconds',
        color='Backend Engine',
        barmode='group',
        text='Iteration Duration',
        log_y=True,
        # template ='ggplot2',
    )
    fig.update_layout(
        title_x=0.5,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1,
            x=0.5,
            xanchor='center',
        ),
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=True),
        # paper_bgcolor='rgba(0,0,0,0)',
        # plot_bgcolor='rgba(0,0,0,0)',
        bargap=0.30,
        bargroupgap=0.10,
    )
    fig.update_traces(
        textfont_size=14,
        textangle=-60, 
        textposition='auto', 
        cliponaxis=False,
    )
    fig.show()